In [ ]:
CATEGORIES = ["action", "adventure", "animation", "biography", "comedy", "crime", "documentary", "drama", "family",
              "fantasy", "film_noir", "history", "horror", "music", "musical", "mystery", "romance", "sci_fi",
              "short", "sport", "superhero", "thriller", "war", "western"]

In [ ]:
from transformers import ConvNextFeatureExtractor, ConvNextForImageClassification
import torch
from datasets import load_dataset

dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

feature_extractor = ConvNextFeatureExtractor.from_pretrained("facebook/convnext-tiny-224")
model = ConvNextForImageClassification.from_pretrained("facebook/convnext-tiny-224")

inputs = feature_extractor(image, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# model predicts one of the 1000 ImageNet classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label]),

In [ ]:
inputs.values()

In [ ]:
from datasets import load_dataset

# load cifar10 (only small portion for demonstration purposes) 
train_ds, test_ds = load_dataset('cifar10', split=['train[:5000]', 'test[:2000]'])
# split up training into training + validation
splits = train_ds.train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']

In [ ]:
import pandas as pd
df = pd.read_csv('data/preprocessed.csv')
df.head(5)

In [ ]:
model_name_or_path = "facebook/convnext-tiny-224"

In [ ]:
feature_extractor = ConvNextFeatureExtractor.from_pretrained(model_name_or_path)

In [ ]:
import torch
import torchvision.transforms as transforms
import os
import pandas as pd
from skimage import io
from torch.utils.data import (
    Dataset,
    DataLoader
)

class FilmFeatureExtractor(object):
    def __call__(self, image, targets):
        sample = feature_extractor(image, return_tensors='pt')
        sample["labels"] = torch.tensor(targets, dtype=torch.float32)
        
        return sample

class FilmDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.shape = self.annotations.shape
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index]['poster_path'])
        image = io.imread(img_path)
        labels = torch.tensor(self.annotations.iloc[index][CATEGORIES])
        data = self.transform(image, labels)
        
        return data

df = FilmDataset(
    csv_file="data/preprocessed.csv",
    root_dir="",
    transform=FilmFeatureExtractor(),
)


In [ ]:
df.__getitem__(0)

## Train test split

In [ ]:
train_size = int(0.4 * df.shape[0])
eval_size = int(0.4 * df.shape[0])
test_size = df.shape[0] - train_size - eval_size

print(df.shape[0], train_size, eval_size, test_size)

In [ ]:
df_train, df_eval, df_test = torch.utils.data.random_split(df, [train_size, eval_size, test_size])

## Model

In [ ]:
model = ConvNextForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(CATEGORIES),
    ignore_mismatched_sizes=True
)

In [ ]:
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./data/img", 
    evaluation_strategy="epoch",
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16
)

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    print(eval_pred)
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
import torch

def collate_fn(batch):

    return {
        'pixel_values': torch.stack([x['pixel_values'][0] for x in batch]),
        'labels': torch.stack([x['labels'] for x in batch])
    }

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=df_train,
    eval_dataset=df_eval,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [88]:
feature_extractor = ConvNextFeatureExtractor.from_pretrained(model_name_or_path)

In [101]:
import torch
import torchvision.transforms as transforms
import os
import pandas as pd
from skimage import io
from torch.utils.data import (
    Dataset,
    DataLoader
)

class FilmFeatureExtractor(object):
    def __call__(self, image, targets):
        sample = feature_extractor(image, return_tensors='pt')
        sample["labels"] = torch.tensor(targets, dtype=torch.float32)
        
        return sample

class FilmDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.shape = self.annotations.shape
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index]['poster_path'])
        image = io.imread(img_path)
        labels = torch.tensor(self.annotations.iloc[index][CATEGORIES])
        data = self.transform(image, labels)
        
        return data

df = FilmDataset(
    csv_file="data/preprocessed.csv",
    root_dir="",
    transform=FilmFeatureExtractor(),
)


In [102]:
df.__getitem__(0)

/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)


{'pixel_values': tensor([[[[-0.5082, -0.5596, -0.5938,  ...,  1.4954,  1.4954,  1.4612],
          [-0.5938, -0.6623, -0.7137,  ...,  1.5639,  1.5468,  1.5468],
          [-0.6452, -0.7137, -0.7650,  ...,  1.5468,  1.4612,  1.5468],
          ...,
          [-1.4158, -1.3815, -1.3987,  ..., -1.4329, -1.5357, -1.6213],
          [-1.4843, -1.4500, -1.4329,  ..., -1.5357, -1.5014, -1.5014],
          [-1.6042, -1.5528, -1.5185,  ..., -1.7412, -1.7925, -1.8097]],

         [[-0.3200, -0.3725, -0.4076,  ...,  1.4832,  1.4657,  1.4307],
          [-0.3725, -0.4426, -0.4951,  ...,  1.5532,  1.5007,  1.5182],
          [-0.4076, -0.4776, -0.5476,  ...,  1.5357,  1.4482,  1.5182],
          ...,
          [-1.1604, -1.1078, -1.1429,  ..., -1.0028, -1.0903, -1.1779],
          [-1.2829, -1.2304, -1.1954,  ..., -1.1078, -1.0903, -1.1253],
          [-1.4405, -1.4055, -1.3529,  ..., -1.3880, -1.4580, -1.4755]],

         [[ 0.5659,  0.5311,  0.5136,  ...,  0.7228,  0.7402,  0.7054],
          [ 0

## Train test split

In [111]:
train_size = int(0.4 * df.shape[0])
eval_size = int(0.4 * df.shape[0])
test_size = df.shape[0] - train_size - eval_size

print(df.shape[0], train_size, eval_size, test_size)

4188 1675 1675 838


In [112]:
df_train, df_eval, df_test = torch.utils.data.random_split(df, [train_size, eval_size, test_size])

## Model

In [142]:
model = ConvNextForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(CATEGORIES),
    ignore_mismatched_sizes=True
)

loading configuration file config.json from cache at /Users/tiginamaria1999/.cache/huggingface/hub/models--facebook--convnext-tiny-224/snapshots/0d1c8dedaa107d4ae537c5b10e5cd0a8c865e84e/config.json
Model config ConvNextConfig {
  "architectures": [
    "ConvNextForImageClassification"
  ],
  "depths": [
    3,
    3,
    9,
    3
  ],
  "drop_path_rate": 0.0,
  "hidden_act": "gelu",
  "hidden_sizes": [
    96,
    192,
    384,
    768
  ],
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23"
  },
  "image_size": 224,
  "ini

In [149]:
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./data/img", 
    evaluation_strategy="epoch",
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [150]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [151]:
def compute_metrics(eval_pred):
    print(eval_pred)
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [152]:
import torch

def collate_fn(batch):

    return {
        'pixel_values': torch.stack([x['pixel_values'][0] for x in batch]),
        'labels': torch.stack([x['labels'] for x in batch])
    }

In [153]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=df_train,
    eval_dataset=df_eval,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

In [154]:
trainer.train()

/usr/local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1675
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 315
  Number of trainable parameters = 27838584
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykern

Epoch,Training Loss,Validation Loss


/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets

/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets

/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets

...

/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets

/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets

/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets

/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets, dtype=torch.float32)
/var/folders/4s/hl9y6lqs3h1c9qxxnbjmtvsr0000gn/T/ipykernel_3675/201434282.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample["labels"] = torch.tensor(targets

FileNotFoundError: No such file: '/Users/tiginamaria1999/PycharmProjects/IndustrialML_ITMO/task2/data/superhero/the_batman_2.jpg'